zasilkovna tabulka:
- zip
- city
- orders_id

In [ ]:
CREATE TEMPORARY TABLE ZASILKOVNA_CZ AS
select
z."zip"
,z."city"
,zo."orders_id"
from "sm_zasilkovna_branches" as z
Left Join "sm_zasilkovna_orders" as zo ON zo."branches_id" = z."id"
where z."country" = 'cz'

CREATE TEMPORARY TABLE BALIKOVNA_CZ AS
select
b."zipcode"
,b."city"
,bo."orders_id"
from "sm_balikovna_branches" as b
Left Join "sm_balikovna_orders" as bo ON bo."branches_id" = b."id"

CREATE TEMPORARY TABLE DPD_CZ AS
select
dpd."postcode" 
,dpd."city"
,dpdo."orders_id"
from "sm_balikovna_branches" as dpd
Left Join "sm_balikovna_orders" as dpdo ON dpdo."branches_id" = dpd."id"

CREATE TEMPORARY  TABLE GEIS_CZ AS
select
g."zipcode"
,g."city"
,go."orders_id"
from "sm_geis_point_branches" as g
Left Join "sm_geis_orders" as go ON go."branches_id" = g."id"
where g."country" = 'CZ'

CREATE TEMPORARY TABLE BALIKNP_CZ AS
select  
"zipcode"
,city
,"orders_id"
from "sm_balik_napostu"

spojim vsechno pomoci unionem a zaroven vytvorim tabulku transport

In [ ]:
CREATE OR REPLACE TABLE TRANSPORTS AS
select
z."zip" AS ZIP
,z."city" AS CITY_T
,zo."orders_id" AS T_ORDERS_ID
from "sm_zasilkovna_branches" as z
Left Join "sm_zasilkovna_orders" as zo ON zo."branches_id" = z."id"
where z."country" = 'cz'

UNION ALL

select
b."zipcode" AS ZIP
,b."city" AS CITY_T
,bo."orders_id" AS T_ORDERS_ID
from "sm_balikovna_branches" AS b
Left Join "sm_balikovna_orders" AS bo ON bo."branches_id" = b."id"

UNION ALL

select
dpd."postcode"  AS ZIP
,dpd."city" AS CITY_T
,dpdo."orders_id" AS T_ORDERS_ID
from "sm_dpd_parcel_shops_branches" AS dpd
Left Join "sm_dpd_parcel_shop_orders" AS dpdo ON dpdo."branches_id" = dpd."id"

UNION ALL

select
g."zipcode" AS ZIP
,g."city" AS CITY_T
,go."orders_id" AS T_ORDERS_ID
from "sm_geis_point_branches" AS g
Left Join "sm_geis_orders" AS go ON go."branches_id" = g."id"

UNION ALL

select   
"zipcode" AS ZIP
,city AS CITY_T
,"orders_id" AS T_ORDERS_ID
from "sm_balik_napostu"


Tabulka objednavky pro ZIP_CODES

Spojim tabulku orders , zipcode stores, transports 


In [ ]:
CREATE OR REPLACE TABLE ORDERS_2 AS
select 
 O2."id" AS ID
,O2."number" AS ORDER_NUMBER
,O2."zipcode" AS ZIP_FROM_ORDER
,O2."city" AS CITY_FROM_ORDER
,O2."delivery_zipcode" AS DELIVERY_ZIP_ORDER
,O2."delivery_city" AS DELIVERY_CITY_ORDER
,ZCS.zip_code   -- bude nase hlavni sloupce na psc
,T.ZIP AS ZIP_T
,T.CITY_T AS CITY_T
,O2."shipping_methods_name_cs" AS SHIPPING_METHOD
,O2."shipping_methods_id"  AS SHIPPING_METHOD_ID
,SPLIT ("date", '-')[0] AS IYEAR
from orders_2 O2
LEFT JOIN zip_code_stores AS ZCS ON ZCS.id_prodejny = O2."shipping_methods_id"
LEFT JOIN transports AS T ON T.t_orders_id = O2."id"

--pridame sloupec carrier_name

ALTER TABLE ORDERS_2
ADD COLUMN CARRIER_NAME VARCHAR (30);

-- a doplnime sloupec

UPDATE ORDERS_2
SET CARRIER_NAME = 
  CASE 
    WHEN "SHIPPING_METHOD" LIKE '%GLS%' THEN 'GLS' 
    WHEN "SHIPPING_METHOD"  LIKE '%DPD%' THEN 'DPD' 
    WHEN "SHIPPING_METHOD"  LIKE '%GEIS%' THEN 'GEIS' 
    WHEN "SHIPPING_METHOD"  LIKE '%PPL%' THEN 'PPL' 
    WHEN "SHIPPING_METHOD"  LIKE '%Balíkovna%' THEN 'BALIKOVNA' 
    WHEN "SHIPPING_METHOD"  LIKE '%iftago%' THEN 'LIFTAGO' 
    WHEN "SHIPPING_METHOD"  LIKE '%Česká%' THEN 'ČESKÁ POŠTA' 
    WHEN "SHIPPING_METHOD"  LIKE '%loženka%' THEN 'ULOŽENKA' 
    WHEN "SHIPPING_METHOD"  LIKE '%odo%' THEN 'DODO' 
    WHEN "SHIPPING_METHOD"  LIKE '%Packeta%' THEN 'PACKETA' 
    WHEN "SHIPPING_METHOD"  LIKE '%silkovna%' THEN 'PACKETA' 
    WHEN "SHIPPING_METHOD"  LIKE '%DHL%' THEN 'DHL'
    WHEN "SHIPPING_METHOD"  LIKE '%SPS%' THEN 'SPS - SK'
    WHEN "SHIPPING_METHOD"  LIKE '%venská po%' THEN 'SK pošta'
    WHEN "SHIPPING_METHOD"  LIKE '%test%' THEN 'test'
    WHEN "SHIPPING_METHOD"  LIKE '%UPS%' THEN 'UPS'
    WHEN "SHIPPING_METHOD"  LIKE '%Bratislava - Osobní odběr%' THEN 'Bratislava - Osobní odběr'
    ELSE 'OSOBNÍ ODBĚR' 
  END;

a nasledne sloupec ZIp_CODE updatuju

In [ ]:
--- updatuju zip_code from tabulky TRANSPORT

UPDATE ORDERS_4
SET zip_code = ZIP_T
WHERE zip_code IS NULL ;

UPDATE ORDERS_4
SET zip_code = ZIP_T
WHERE zip_code = '' ;

-- updatuju zip_code from tabulky orders, ALE beru DELIVERY_ZIP

UPDATE ORDERS_4
SET zip_code = DELIVERY_ZIP_ORDER
WHERE zip_code IS NULL ;

UPDATE ORDERS_4
SET zip_code = DELIVERY_ZIP_ORDER
WHERE zip_code  = ''  ;

-- u zbytku updatuju zip_code from tabulky orders, zip co zadavaji zakaznici

UPDATE ORDERS_4
SET zip_code = ZIP_FROM_ORDER
WHERE zip_code = '' ;

--- Bratislava osobni odber smazem - 134 zaznamu
DELETE FROM ORDERS_4 
WHERE SHIPPING_METHOD = 'Bratislava - Osobní odběr'
--- zustalo pak 9 zaznamu bez zip_code
DELETE FROM ORDERS_4 
WHERE id IN 
('9455',
'13467',
'41764',
'51641',
'60314',
'74451',
'93735',
'10701',
'27322',
'273229',
'107017')


chtela jsem jeste pridat NAZOBCE a NAZOKRESU ale milion radku jsem dostala 13 milion

In [ ]:
select 
-- DISTINCT o4.id
 o4.id
,o4.order_number
,o4.zip_code
,zcc.psc
,o4.city_from_order
,o4.delivery_city_order
,o4.city_t
,zcc.nazobce
,zcc.nazokresu
from orders_4 o4
LEFT JOIN zip_code_city zcc ON zcc.psc = o4.zip_code

